In [1]:
#import library
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import folium
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from numba import jit, cuda

from timeit import default_timer as timer  

%matplotlib inline


In [2]:
DevType = pd.read_csv("DevType.csv")
Salary = pd.read_csv ("Salary.csv")
df = pd.read_csv ("2021-survey_results_public.csv")

In [3]:
df.head(2)

,ResponseId,MainBranch,Employment,Country,iso_alpha,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,...,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Slovakia,SVK,NaN,NaN,"Secondary school (e.g. American high school, G...",18 - 24 years,Coding Bootcamp;Other online resources (ex: vi...,...,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,62268.0
1,2,I am a student who is learning to code,"Student, full-time",Netherlands,NLD,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",...,18-24 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,NaN


In [4]:
Standart_DF = df[['MainBranch' , 'Employment' , 'Country' , 'EdLevel' , 'YearsCodePro' , 'Age' , 'iso_alpha']].reset_index()
Standart_DF.head(3)

,index,MainBranch,Employment,Country,EdLevel,YearsCodePro,Age,iso_alpha
0,0,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Slovakia,"Secondary school (e.g. American high school, G...",NaN,25-34 years old,SVK
1,1,I am a student who is learning to code,"Student, full-time",Netherlands,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",NaN,18-24 years old,NLD
2,2,"I am not primarily a developer, but I write co...","Student, full-time",Russian Federation,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",NaN,18-24 years old,RUS


In [5]:
Salary.head(3)

,Unnamed: 0,index,TotalByYearUSD,Group
0,17,37,300000,Group1
1,26,64,500000,Group1
2,43,96,545717,Group1


In [6]:
DevType.head(3)

,Unnamed: 0,index,DevType
0,0,0,"Developer, mobile"
1,34,34,"Developer, mobile"
2,56,56,"Developer, mobile"


In [7]:
#Lets merge our dataset
salary_DevType = pd.merge (DevType , Salary ,on = 'index')
salary_DevType_df = pd.merge (salary_DevType , Standart_DF , on = 'index' ).rename (columns = {'Type':'DevType'})[
    ['index' , 'DevType' , 'TotalByYearUSD' , 'Group' ,'MainBranch' ,'Employment' ,'Country' ,'EdLevel' ,'YearsCodePro','Age','iso_alpha' ]]

In [8]:
#Cleaning The Years Code Pro Columns
#Drop na
salary_DevType_df.dropna (subset = ['YearsCodePro'] , axis = 0 , inplace = True)

In [9]:
# Replace Object values
salary_DevType_df['YearsCodePro'].replace ('Less than 1 year' , '1' , inplace = True)
salary_DevType_df['YearsCodePro'].replace ('More than 50 years' , '50' , inplace = True)


In [10]:
#Convert type from object to int 
salary_DevType_df['YearsCodePro'] = salary_DevType_df['YearsCodePro'].astype (int)

In [11]:
salary_DevType_df['YearsCodePro'].describe()

count    119175.000000
mean          9.995704
std           8.288706
min           1.000000
25%           4.000000
50%           7.000000
75%          14.000000
max          50.000000
Name: YearsCodePro, dtype: float64

In [20]:
buble = salary_DevType_df[['DevType' , 'TotalByYearUSD' , 'YearsCodePro' ,'index']].groupby(['DevType' ,'YearsCodePro']).agg(
    {'TotalByYearUSD'  : 'mean' , 'index' : 'count'}).reset_index().rename(
    columns = {'index':'count'}).sort_values(by = 'count'  , ascending = False)


buble



fig = px.scatter(buble, x="TotalByYearUSD", y="YearsCodePro", animation_frame="DevType",
           size="TotalByYearUSD", color="TotalByYearUSD"  )#, range_x=[duble['TotalByYearUSD'].min()   , duble['TotalByYearUSD'].max()])

fig.update_layout( showlegend=True , title_text = 'Years Code Pro ,Total salary Year USD and DevType '  ) 


fig.show()
fig.write_html('first_figure.html', auto_open=True)


In [13]:
Table = buble[['DevType','TotalByYearUSD','count']].groupby('DevType').agg({'TotalByYearUSD':'mean','count':'sum'}
                                                                        ).reset_index().sort_values(by = 'TotalByYearUSD',ascending = False)

Table


,DevType,TotalByYearUSD,count
17,Engineering manager,136845.192615,2682
22,"Senior Executive (C-Suite, VP, etc.)",131697.021543,1259
16,"Engineer, site reliability",121516.560796,1781
19,Other (please specify):,117149.570671,1882
5,DevOps specialist,114663.294094,5318
15,"Engineer, data",113300.925605,2980
7,"Developer, back-end",111510.217616,21024
12,"Developer, game or graphics",110036.168314,1266
11,"Developer, full-stack",108895.835372,23715
21,Scientist,107390.523751,1301


# Map

In [18]:

da = salary_DevType_df [['DevType' , 'TotalByYearUSD' , 'Country' , 'iso_alpha']]
map_data = da.groupby(['Country' , 'DevType' , 'iso_alpha']).mean().reset_index().sort_values(by = 'TotalByYearUSD' , ascending = False)


fig = px.choropleth (map_data , locations = 'iso_alpha' , color = 'TotalByYearUSD'  , hover_name =  'Country' ,
                    animation_frame="DevType")

fig.update_layout( showlegend=True , title_text = 'Geographical distribution ,Total salary Year USD and DevType '  ) 


fig.show()
fig.write_html('first_figure.html', auto_open=True)

map_data


,Country,DevType,iso_alpha,TotalByYearUSD
645,Cyprus,Scientist,CYP,682147.0
191,Bahrain,"Senior Executive (C-Suite, VP, etc.)",BHR,682147.0
628,Cyprus,Data scientist or machine learning specialist,CYP,682147.0
640,Cyprus,"Engineer, data",CYP,682147.0
2244,San Marino,"Developer, full-stack",SMR,627575.0
...,...,...,...,...
1487,Liberia,"Developer, desktop or enterprise applications",LBR,11.0
1486,Liberia,"Developer, back-end",LBR,11.0
1484,Liberia,Database administrator,LBR,11.0
1483,Liberia,Data or business analyst,LBR,11.0


In [15]:


da = salary_DevType_df [['DevType' , 'TotalByYearUSD' , 'Country']]
group = da.groupby(['Country' , 'DevType']).mean().reset_index().sort_values(by = 'TotalByYearUSD' , ascending = False)
group

colors = ['#9e0142', '#d53e4f', '#f46d43', '#fdae61' , '#fee08b', '#e6f598', '#abdda4', '#66c2a5', '#3288bd', '#5e4fa2']


fig = go.Figure()

DevType_list = list(group['DevType'].unique())

for DevType in DevType_list:
    fig.add_trace( go.Bar(  x = group['Country'][group['DevType']==DevType][0:40],
                                 y = group ['TotalByYearUSD'][group ['DevType']==DevType],
                                    marker=dict(color=colors , colorscale='viridis'  )     
                         ) )
    
buttons = []

for i, DevType in enumerate(DevType_list):
    args = [False] * len(DevType_list)
    args[i] = True
    button = dict(label = DevType, method = "update",  args=[{"visible": args}])
    buttons.append(button)
    
fig.update_layout(updatemenus=[dict(active=0,type="dropdown",buttons=buttons,x = 0,y=1.1,xanchor='left',yanchor='bottom')], 
    autosize=False,
    width=1000,
    height=600)
